# Terrain Visualization with Matplotlib Colormaps

This notebook demonstrates end-to-end terrain rendering using matplotlib colormap integration.
It showcases the adapter system between forge3d and matplotlib for seamless colormap usage.

**Expected runtime:** < 5 minutes  
**Memory usage:** < 256 MiB  
**Outputs:** terrain_matplotlib.png

In [ ]:
import numpy as np
import sys
import os
import time
from pathlib import Path

# Add repo root to path for imports
if Path('../..').exists():
    sys.path.insert(0, str(Path('../..').resolve()))

try:
    import forge3d as f3d
    print(f"✓ forge3d {f3d.__version__} loaded successfully")
except ImportError as e:
    print(f"✗ Failed to import forge3d: {e}")
    print("Please run: maturin develop --release")
    sys.exit(1)

# Check matplotlib availability
try:
    import matplotlib as mpl
    from forge3d.adapters import matplotlib_to_forge3d_colormap, is_matplotlib_available
    print(f"✓ matplotlib {mpl.__version__} integration available: {is_matplotlib_available()}")
except ImportError:
    print("✗ matplotlib not available - some features will be skipped")
    mpl = None

## Device Information

Check GPU capabilities and adapter information for the rendering pipeline.

In [ ]:
# Device capabilities check
try:
    device_info = f3d.device_probe()
    print("🖥️  Device Information:")
    print(f"   Backend: {device_info.get('backend', 'unknown')}")
    print(f"   Adapter: {device_info.get('adapter_name', 'unknown')}")
    
    features = device_info.get('features', '')
    if 'TIMESTAMP_QUERY' in features:
        print("   ✓ GPU timing available")
    else:
        print("   ⚠ GPU timing fallback to CPU")
        
    # Memory budget check
    print(f"\n📊 Starting notebook with memory budget compliance")
    
except Exception as e:
    print(f"⚠ Device probe failed: {e}")
    print("Continuing with fallback adapter...")

## Synthetic Terrain Generation

Generate a procedural heightfield for terrain visualization.

In [ ]:
# Generate synthetic terrain data
np.random.seed(42)  # Deterministic results
size = 256
x = np.linspace(-2, 2, size)
y = np.linspace(-2, 2, size)
X, Y = np.meshgrid(x, y)

# Create realistic terrain with multiple elevation features
terrain = (
    np.exp(-(X**2 + Y**2)) * 100 +  # Central peak
    np.exp(-((X-1)**2 + (Y+1)**2) / 0.5) * 60 +  # Secondary peak
    np.sin(X * 3) * np.cos(Y * 2) * 20 +  # Ridges
    np.random.normal(0, 2, (size, size))  # Noise
)

# Ensure contiguous float32 for GPU upload
terrain = np.ascontiguousarray(terrain, dtype=np.float32)

print(f"🏔️  Generated terrain: {terrain.shape} ({terrain.dtype})")
print(f"   Height range: {terrain.min():.1f} to {terrain.max():.1f}")
print(f"   Memory footprint: {terrain.nbytes / 1024:.1f} KB")
print(f"   Array contiguous: {terrain.flags.c_contiguous}")

## Matplotlib Colormap Integration

Demonstrate forge3d's matplotlib adapter for colormap conversion.

In [ ]:
if mpl:
    # Test multiple matplotlib colormaps
    colormaps = ['viridis', 'terrain', 'plasma', 'coolwarm']
    
    print("🎨 Converting matplotlib colormaps:")
    for cmap_name in colormaps:
        try:
            forge_cmap = matplotlib_to_forge3d_colormap(cmap_name)
            print(f"   ✓ {cmap_name}: {forge_cmap.shape} RGBA values")
        except Exception as e:
            print(f"   ✗ {cmap_name}: {e}")
    
    # Use terrain colormap for this demo
    selected_colormap = 'terrain'
    print(f"\n🎯 Using colormap: {selected_colormap}")
else:
    print("⚠ Skipping matplotlib integration - using built-in colormap")
    selected_colormap = 'viridis'

## Terrain Rendering Pipeline

Render terrain using forge3d with GPU acceleration and timing measurements.

In [ ]:
# Initialize renderer
start_time = time.time()

try:
    renderer = f3d.Renderer(512, 512, prefer_software=False)
    print(f"✓ Renderer initialized: {renderer.info()}")
    
    # Upload terrain data
    upload_start = time.time()
    renderer.upload_height_r32f(terrain, spacing=0.1, exaggeration=1.5)
    upload_time = (time.time() - upload_start) * 1000
    print(f"✓ Height data uploaded: {upload_time:.1f} ms")
    
    # Configure terrain visualization
    height_range = [terrain.min(), terrain.max()]
    renderer.set_height_range(height_range[0], height_range[1])
    
    # Set camera for good terrain view
    renderer.set_camera(
        eye=(0.0, 150.0, -200.0),
        target=(0.0, 0.0, 0.0),
        up=(0.0, 1.0, 0.0)
    )
    
    # Configure lighting
    renderer.set_sun(elevation_deg=45.0, azimuth_deg=315.0)  # From northwest
    renderer.set_exposure(1.2)
    
    print(f"✓ Terrain configured: range {height_range[0]:.1f} to {height_range[1]:.1f}")
    
except Exception as e:
    print(f"✗ Renderer setup failed: {e}")
    raise

In [ ]:
# Render with timing
try:
    render_start = time.time()
    
    # Render to RGBA array
    rgba_array = renderer.render_rgba()
    render_time = (time.time() - render_start) * 1000
    
    print(f"✓ Terrain rendered: {rgba_array.shape} in {render_time:.1f} ms")
    
    # Save output PNG
    output_path = "terrain_matplotlib.png"
    save_start = time.time()
    f3d.numpy_to_png(output_path, rgba_array)
    save_time = (time.time() - save_start) * 1000
    
    print(f"✓ PNG saved: {output_path} ({save_time:.1f} ms)")
    
    # Verify file was created
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path) / 1024
        print(f"✓ File verified: {file_size:.1f} KB")
    else:
        raise FileNotFoundError(f"Output file {output_path} not created")
        
except Exception as e:
    print(f"✗ Rendering failed: {e}")
    raise

## Performance Metrics

Analyze rendering performance and memory usage.

In [ ]:
# Performance summary
total_time = (time.time() - start_time) * 1000

print("📊 Performance Summary:")
print(f"   Total notebook time: {total_time:.1f} ms")
print(f"   Terrain upload: {upload_time:.1f} ms")
print(f"   GPU rendering: {render_time:.1f} ms")
print(f"   PNG encoding: {save_time:.1f} ms")

# Calculate throughput
pixels_rendered = 512 * 512
megapixels_per_sec = (pixels_rendered / 1_000_000) / (render_time / 1000)
print(f"   Throughput: {megapixels_per_sec:.2f} MP/s")

# Memory analysis
terrain_mb = terrain.nbytes / (1024 * 1024)
rgba_mb = rgba_array.nbytes / (1024 * 1024)
total_mb = terrain_mb + rgba_mb

print(f"\n💾 Memory Usage:")
print(f"   Terrain data: {terrain_mb:.2f} MB")
print(f"   RGBA output: {rgba_mb:.2f} MB")
print(f"   Total arrays: {total_mb:.2f} MB")
print(f"   Budget compliance: {'✓' if total_mb < 256 else '⚠'} (<256 MB target)")

# Adapter integration status
if mpl:
    print(f"\n🔗 Integration Status:")
    print(f"   Matplotlib adapter: ✓ Active")
    print(f"   Colormap: {selected_colormap}")
    print(f"   Terrain range mapping: {height_range[0]:.1f} → {height_range[1]:.1f}")

## Output Validation

Verify the rendered output meets quality expectations.

In [ ]:
# Validate output quality
print("🔍 Output Validation:")

# Check image dimensions
expected_shape = (512, 512, 4)
if rgba_array.shape == expected_shape:
    print(f"   ✓ Dimensions: {rgba_array.shape}")
else:
    print(f"   ✗ Dimensions: got {rgba_array.shape}, expected {expected_shape}")

# Check data range
if rgba_array.dtype == np.uint8 and rgba_array.min() >= 0 and rgba_array.max() <= 255:
    print(f"   ✓ Data range: [{rgba_array.min()}, {rgba_array.max()}] uint8")
else:
    print(f"   ✗ Data range: [{rgba_array.min()}, {rgba_array.max()}] {rgba_array.dtype}")

# Check for non-zero content (not blank image)
non_zero_pixels = np.count_nonzero(rgba_array[:,:,:3])  # RGB channels
total_rgb_pixels = rgba_array.shape[0] * rgba_array.shape[1] * 3
content_ratio = non_zero_pixels / total_rgb_pixels

if content_ratio > 0.1:  # At least 10% non-zero content
    print(f"   ✓ Content: {content_ratio:.1%} non-zero pixels")
else:
    print(f"   ⚠ Content: {content_ratio:.1%} non-zero pixels (may be too dark)")

# Alpha channel check
alpha_values = np.unique(rgba_array[:,:,3])
if len(alpha_values) == 1 and alpha_values[0] == 255:
    print(f"   ✓ Alpha: Fully opaque")
else:
    print(f"   ⚠ Alpha: {len(alpha_values)} unique values {alpha_values[:5]}")

print(f"\n✅ Notebook completed successfully!")
print(f"📁 Output: {output_path} ({file_size:.1f} KB)")
if total_time < 300000:  # 5 minutes = 300,000 ms
    print(f"⏱️  Runtime: {total_time/1000:.1f}s (within 5min budget)")
else:
    print(f"⚠️ Runtime: {total_time/1000:.1f}s (exceeded 5min budget)")